<a href="https://colab.research.google.com/github/gnitnaw/Programs/blob/master/Chapter7/svmquad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!python --version
!rm svmcirc.csv
!wget https://raw.github.com/gnitnaw/Programs/master/Chapter7/svmcirc.csv

Python 3.7.12
rm: cannot remove 'svmcirc.csv': No such file or directory
--2022-02-25 10:40:19--  https://raw.github.com/gnitnaw/Programs/master/Chapter7/svmcirc.csv
Resolving raw.github.com (raw.github.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.github.com (raw.github.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/gnitnaw/Programs/master/Chapter7/svmcirc.csv [following]
--2022-02-25 10:40:19--  https://raw.githubusercontent.com/gnitnaw/Programs/master/Chapter7/svmcirc.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 994 [text/plain]
Saving to: ‘svmcirc.csv’

svmcirc.csv         100%[===================>]     994  --.-KB/

# Example 7.6
## Import modules

In [6]:
import numpy as np
from numpy import genfromtxt
from sklearn.svm import SVC
print(np.__version__)

1.21.5


## Set random seed

In [7]:
np.random.seed(12345)

## Original Program

In [8]:
data = genfromtxt('svmcirc.csv', delimiter=',') 
x = data[:,[0,1]] #vectors are rows
y = data[:,[2]].reshape(len(x),) #labels

tmp = np.sum(np.power(x,2),axis=1).reshape((len(x),1))
z = np.hstack((x,tmp))

clf = SVC(C = np.inf, kernel='linear')
clf.fit(z,y)

print("Support Vectors \n", clf.support_vectors_)
print("Support Vector Labels ",y[clf.support_])
print("Alpha ",clf.dual_coef_)
print("Bias ",clf.intercept_)

Support Vectors 
 [[ 0.038758    0.53796     0.29090314]
 [-0.49116    -0.20563     0.28352184]
 [-0.45068    -0.04797     0.20541358]
 [-0.061107   -0.41651     0.17721465]]
Support Vector Labels  [-1. -1.  1.  1.]
Alpha  [[ -46.49249413 -249.01807328  265.31805855   30.19250886]]
Bias  [5.617891]
